# Zillow Time Series

In [1]:
#Exploratory 
import pandas as pd
import numpy as np

#Data Visualization 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns



#Data Preprocesing

#Data Modeling  
from statsmodels.tsa.arima_model import ARMA
import statsmodels.api as sm


#Data Evaluation 
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


# Ignore any warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import data
df = pd.read_csv('Data/zillow_data.csv')
df

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14718,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,94600.0,94300.0,94000.0,...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Claremont,Grafton,14720,92700.0,92500.0,92400.0,...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Richmond,Madison,14721,57100.0,57300.0,57500.0,...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,191100.0,192400.0,193700.0,...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400


### LOL why RegionName and CountyName ??

# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [3]:
# Renaming columns 

df.rename(columns={'RegionName': 'Zipcode',
                       'CountyName':'County'},
              inplace=True)

In [4]:
df.head(2)

,RegionID,Zipcode,City,State,Metro,County,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800


### Choose a State to focus on

In [5]:
df['State'].value_counts().head(10)

CA    1224
NY    1015
TX     989
PA     831
FL     785
OH     588
IL     547
NJ     502
MI     499
IN     428
Name: State, dtype: int64

## Ask Aidan CA NY or TX
- I want NY

In [6]:
df = df[df['State']=='NY']
df.isna().sum()

RegionID     0
Zipcode      0
City         0
State        0
Metro       52
            ..
2017-12      0
2018-01      0
2018-02      0
2018-03      0
2018-04      0
Length: 272, dtype: int64

In [7]:
df['Metro']

6        New York
10       New York
12       New York
13       New York
20       New York
           ...   
14663      Albany
14693      Albany
14702    Kingston
14715    Kingston
14717         NaN
Name: Metro, Length: 1015, dtype: object

# Step 2: Data Preprocessing

In [8]:
# def get_datetimes(df):
#     """
#     Takes a dataframe:
#     returns only those column names that can be converted into datetime objects 
#     as datetime objects.
#     NOTE number of returned columns may not match total number of columns in passed dataframe
#     """
    
#     return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

### Creating Statistical Columns

- Create ROI, Standard Deviation, Mean, and Coefficient of Variance statistical columns

In [9]:
#Calculate historical return on investment
df['ROI']= ((df['2018-04']/df['1996-04'])-1) *100 

#Calculate standard deviation of monthly values
df['std']=df.loc[:,'1996-04':'2018-04'].std(skipna=True, axis=1)

#Calculate historical mean value
df['mean']=df.loc[:,'1996-04':'2018-04'].mean(skipna=True, axis=1)


In [10]:
df[['Zipcode','std','mean','ROI']].head()

,Zipcode,std,mean,ROI
6,10467,8.569914e+04,2.923392e+05,173.315893
10,11226,2.080187e+05,4.614242e+05,494.567901
12,11375,2.240221e+05,6.081170e+05,329.714739
13,11235,1.665122e+05,4.771932e+05,328.451444
20,10011,2.650867e+06,7.755844e+06,NaN


In [11]:
df

,RegionID,Zipcode,City,State,Metro,County,SizeRank,1996-04,1996-05,1996-06,...,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,ROI,std,mean
6,61807,10467,New York,NY,New York,Bronx,7,152900.0,152700.0,152600.0,...,411600,413200,414300,413900,411400,413200,417900,173.315893,8.569914e+04,2.923392e+05
10,62037,11226,New York,NY,New York,Kings,11,162000.0,162300.0,162600.0,...,870000,885100,887800,890500,901700,930700,963200,494.567901,2.080187e+05,4.614242e+05
12,62087,11375,New York,NY,New York,Queens,13,252400.0,251800.0,251400.0,...,1066400,1081200,1088800,1092700,1089500,1084000,1084600,329.714739,2.240221e+05,6.081170e+05
13,62045,11235,New York,NY,New York,Kings,14,190500.0,191000.0,191500.0,...,793900,796000,799700,806600,810600,813400,816200,328.451444,1.665122e+05,4.771932e+05
20,61625,10011,New York,NY,New York,New York,21,NaN,NaN,NaN,...,12050100,12016300,11946500,11978100,11849300,11563000,11478300,NaN,2.650867e+06,7.755844e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14663,62430,12153,Sand Lake,NY,Albany,Rensselaer,14664,95700.0,95000.0,94300.0,...,193100,195400,198300,200500,202600,205200,207300,116.614420,4.097207e+04,1.591992e+05
14693,62433,12156,Schodack,NY,Albany,Rensselaer,14694,94400.0,94600.0,94800.0,...,190500,191100,192000,193600,196100,198000,199400,111.228814,3.625277e+04,1.544072e+05
14702,62599,12480,Shandaken,NY,Kingston,Ulster,14703,70900.0,70400.0,70000.0,...,160100,159900,159000,157000,157700,161000,163700,130.888575,4.250360e+04,1.375313e+05
14715,62556,12429,Esopus,NY,Kingston,Ulster,14716,78300.0,78300.0,78200.0,...,171000,170700,171300,172400,173600,175800,177500,126.692209,4.475911e+04,1.493464e+05


In [12]:
#we dont need
df.drop(['RegionID', 'SizeRank', 'Metro'], axis=1,inplace=True)


In [13]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['Zipcode', 'City', 'State', 'County','ROI','std','mean'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

In [14]:
df = melt_data(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265015 entries, 0 to 268974
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   Zipcode  265015 non-null  int64         
 1   City     265015 non-null  object        
 2   State    265015 non-null  object        
 3   County   265015 non-null  object        
 4   ROI      261025 non-null  float64       
 5   std      265015 non-null  float64       
 6   mean     265015 non-null  float64       
 7   time     265015 non-null  datetime64[ns]
 8   value    265015 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(3)
memory usage: 20.2+ MB


In [15]:
df

,Zipcode,City,State,County,ROI,std,mean,time,value
0,10467,New York,NY,Bronx,173.315893,8.569914e+04,2.923392e+05,1996-04-01,152900.0
1,11226,New York,NY,Kings,494.567901,2.080187e+05,4.614242e+05,1996-04-01,162000.0
2,11375,New York,NY,Queens,329.714739,2.240221e+05,6.081170e+05,1996-04-01,252400.0
3,11235,New York,NY,Kings,328.451444,1.665122e+05,4.771932e+05,1996-04-01,190500.0
5,10128,New York,NY,New York,100.902984,1.470204e+06,5.085436e+06,1996-04-01,3676700.0
...,...,...,...,...,...,...,...,...,...
268970,12153,Sand Lake,NY,Rensselaer,116.614420,4.097207e+04,1.591992e+05,2018-04-01,207300.0
268971,12156,Schodack,NY,Rensselaer,111.228814,3.625277e+04,1.544072e+05,2018-04-01,199400.0
268972,12480,Shandaken,NY,Ulster,130.888575,4.250360e+04,1.375313e+05,2018-04-01,163700.0
268973,12429,Esopus,NY,Ulster,126.692209,4.475911e+04,1.493464e+05,2018-04-01,177500.0


# Step 3: EDA and Visualization

In [16]:
# font = {'family' : 'normal',
#         'weight' : 'bold',
#         'size'   : 22}


# #plt.rc('font', **font)
# matplotlib.rc('font', **font)

# # NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!
# #plt.gcf().autofmt_xdate()





In [17]:
def stationarity_check(df):
    roll_mean = df.rolling(window=12, center=False).mean()
    roll_std = df.rolling(window=12, center=False).std()
    
    df_test = adfuller(df)
    print('Results of Dickey-Fuller Test: \n')

    dfoutput = pd.Series(df_test[0:4], index=['Test Statistic', 'p-value', 
                                             '#Lags Used', 'Number of Observations Used'])
    for key,value in df_test[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    roll_mean.plot()
    roll_std.plot()
    return df_test

### I couldnt run those functions ¯\_(ツ)_/¯ 

# Step 4: Reshape from Wide to Long Format

In [18]:
# def melt_data(df):
#     """
#     Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
#     Returns a long-form datetime dataframe 
#     with the datetime column names as the index and the values as the 'values' column.
    
#     If more than one row is passes in the wide-form dataset, the values column
#     will be the mean of the values from the datetime columns in all of the rows.
#     """
    
#     melted = pd.melt(df, id_vars=['Zipcode', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'County'], var_name='time')
#     melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
#     melted = melted.dropna(subset=['value'])
#     return melted.groupby('time').aggregate({'value':'mean'})

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results